In [2]:
import os
import numpy as np
import path

#WIP module for deep system identification: (deepSI)
import deepSI
from deepSI import System_data_list, System_data
from deepSI.utils import fit_with_early_stopping
from deepSI.fit_systems import SS_encoder, SS_encoder_deriv_general #models

## CT CED

In [ ]:
dataset = 'CED'
norm_list = np.geomspace(0.0001, 10, num=21)

if dataset=='CED':
    nx = 3
    na = nb = 5
    batch_size = 32
    epochs_per_early_stopping_check = 300 #minimum epochs
    nf = 30 #T in paper (nf = number of future steps)

    data_full = deepSI.datasets.CED()
    train = System_data_list([data_i[:300] for data_i in data_full])
    test = System_data_list([data_i[300:] for data_i in data_full])
    val = System_data_list([t[:100] for t in test])

elif dataset=='CCT':
    nx = 2
    na = nb = 4
    batch_size = 64 #there is a bit more data 
    epochs_per_early_stopping_check = 300 #minimum epochs
    nf = 60 #T in paper (nf = number of future steps)

    train, test = deepSI.datasets.Cascaded_Tanks()
    val, test = test[:len(test)//2], test

fit_kwargs = dict(train_sys_data=train, val_sys_data=val, batch_size=batch_size, \
                  epochs=epochs_per_early_stopping_check, verbose=0, loss_kwargs=dict(nf=nf))

for i in range(1000,1001):
    #discrate time
    name = f'./models-encoder-CT-{dataset}/dis-{i}'
    print(f'&&&&&&&&& {name} &&&&&&&&&&&&&&&')
    if not os.path.exists(name):
        sys = SS_encoder(nx=nx, na=na, nb=nb)
        fit_with_early_stopping(sys, fit_kwargs=fit_kwargs)
        sys.save_system(name)
        sys.save_system(name+'-best')
        sys.checkpoint_load_system('_last')
        sys.save_system(name+'-last')
    
    for f_norm in norm_list:
        name = f'./models-encoder-CT-{dataset}/con-{i}-{f_norm:.5f}'
        print(f'&&&&&&&&& {name} &&&&&&&&&&&&&&&')
        if not os.path.exists(name):
            sys = SS_encoder_deriv_general(nx=nx, na=na, nb=nb, f_norm=f_norm)
            fit_with_early_stopping(sys, fit_kwargs=fit_kwargs)
            sys.save_system(name)
            sys.save_system(name+'-best')
            sys.checkpoint_load_system('_last')
            sys.save_system(name+'-last')

&&&&&&&&& ./models-encoder-CT-CED/dis-1000 &&&&&&&&&&&&&&&
Starting training with early stopping with settings: stop_frac=66.67% step0=3,000 max_step=None

######## Early stopping iteration check: 1 ########
	 epochs done: 300.0 steps done: 4,800 last best val loss at step: 4,464
	 Current val: 0.449092 Lowest val: 0.437440
	 stopping condition: (stop_frac*step0+best_id)/last_id=134.667% < 66.667% = stop_frac

######## Early stopping iteration check: 2 ########
	 epochs done: 600.0 steps done: 9,600 last best val loss at step: 5,648
	 Current val: 0.525420 Lowest val: 0.422011
	 stopping condition: (stop_frac*step0+best_id)/last_id=79.667% < 66.667% = stop_frac

######## Early stopping iteration check: 3 ########
	 epochs done: 900.0 steps done: 14,400 last best val loss at step: 5,648
	 Current val: 0.561240 Lowest val: 0.422011
	 stopping condition: (stop_frac*step0+best_id)/last_id=53.111% < 66.667% = stop_frac

&&&&&&&&& ./models-encoder-CT-CED/con-1000-0.00010 &&&&&&&&&&&&&&&
Star